In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Importar Datasets
users_original = pd.read_pickle('users.pkl')
users_featnue = pd.read_csv('features_generadas.csv')

In [3]:
# Quedarnos solo con las columnas utilizables
users_original = users_original[['Cat','bot','default_profile', 'description', 'geo_enabled', 'lang', 'listed_count', 'location', 'name', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_banner_url', 'profile_image_url', 'profile_image_url_https', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'screen_name', 'url','favs_tiempo','followers_tiempo', 'statuses_tiempo','friends_tiempo']]
users_featnue.drop(['user_id.1','tweetsCount', 'semanas'], axis=1,inplace=True)
# Rellenar valores vacíos
users_featnue = users_featnue.fillna(0)

In [4]:
# Generar ejes X de features y Normalizarlo
X_o = users_original.drop(columns=['Cat','bot'])
atributos_o = X_o.columns
X_n = users_featnue.drop(columns=['Cat','bot','user_id'])
atributos_n = X_n.columns

In [5]:
# Generar ejes Y de targets y encodear las etiquetas
le = preprocessing.LabelEncoder()
y_o =  le.fit_transform(users_original['Bot'])
y_n =  le.fit_transform(users_featnue['Bot'])

KeyError: 'Bot'

In [ ]:
# Separar entre train y test
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X_o,y_o)
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_n,y_n)

In [ ]:
scaler_o = StandardScaler()
scaler_o.fit(X_train_o)
X_train_o = scaler_o.transform(X_train_o)
X_test_o = scaler_o.transform(X_test_o)
scaler_n = StandardScaler()
scaler_n.fit(X_train_n)
X_train_n = scaler_n.transform(X_train_n)
X_test_n = scaler_n.transform(X_test_n)

In [ ]:
param_trees = {'n_estimators': [50, 100, 200], 
               'max_features': [1, 5, 8, 10, 20], 
               'max_depth': [2, 10, 25, 50, 70, 100,150], 
               'min_samples_leaf':[1, 5, 8, 10, 50]}

In [ ]:
rf = RandomForestClassifier(class_weight='balanced')
kf = StratifiedKFold(n_splits=3, shuffle=True)

In [ ]:
grid_search_rf_o = GridSearchCV(rf, param_grid=param_trees, cv=kf, verbose=1, n_jobs=-1)
grid_search_rf_n = GridSearchCV(rf, param_grid=param_trees, cv=kf, verbose=1, n_jobs=-1)

In [ ]:
# Entrenar modelo original
grid_search_rf_o.fit(X_train_o, y_train_o)

In [ ]:
# Entrenar modelo de nuevas features
grid_search_rf_n.fit(X_train_n, y_train_n)

In [ ]:
# Guardar Best Estimators
model_o = grid_search_rf_o.best_estimator_
print(model_o)
model_n = grid_search_rf_n.best_estimator_
print(model_n)

In [ ]:
# Predecir valores de test
y_pred_o = model_o.predict(X_test_o)
y_pred_n = model_n.predict(X_test_n)

In [ ]:
# Matriz de confusión con Modelo Original
confusion_matrix(y_test_o, y_pred_o)

In [ ]:
# Heatmap con Modelo Original
sns.heatmap(confusion_matrix(y_test_o, y_pred_o))
plt.ylabel('Verdaderos')
plt.xlabel('Predichos');

In [ ]:
# Matriz de confusión con Modelo con Features Nuevas
confusion_matrix(y_test_n, y_pred_n)

In [ ]:
# Heatmap con Modelo con Features Nuevas
sns.heatmap(confusion_matrix(y_test_n, y_pred_n))
plt.ylabel('Verdaderos')
plt.xlabel('Predichos');

In [ ]:
# Scores con Modelo con Features Nuevas
print(classification_report(y_test_o, y_pred_o))

In [ ]:
# Scores con Modelo con Features Nuevas
print(classification_report(y_test_n, y_pred_n))

In [ ]:
# Calcular la importancia de los atributos ORIGINAL
pd.DataFrame({'Atributo':atributos_o,
              'importancia':model_o.feature_importances_}).sort_values('importancia',
                                                                      ascending=False).head(10)

In [ ]:
# Calcular la importancia de los atributos FEATURES NUEVAS
pd.DataFrame({'Atributo':atributos_n,
              'importancia':model_n.feature_importances_}).sort_values('importancia',
                                                                      ascending=False).head(10)

In [ ]:
# Dump Models
pickle.dump(model_o, open('RF-Original', 'wb'))
pickle.dump(model_n, open('RF-FeatNue', 'wb'))